# Data Cleaning with Pandas and Jupyter Notebooks

## Author(s)

- Author1 = {"name": "Wai-Yin Kwan", "affiliation": "Whirl-i-Gig", "email": "wyk@whirl-i-gig.com", "orcid": "orcid"}


## Purpose
This notebook demonstrates how the [Extending Ocean Drilling Pursuits (eODP)](https://eodp.github.io/index.html) project uses Juypter notebooks to clean and standardize 50 years of sedimentary description and microfossil data. This notebook is developed for Python 3.8+ and Jupyter Lab.


## Technical contributions

- demostration of creating a reproducible data cleaning workflow using Git, unit testing, directory organization, and modular code


## Methodology
For eODP, we are cleaning and standardizing over 10,000 CSVs of ocean core data. For this notebook, we are cleaning just 10 files. 

For each data cleaning step, we created separate section and headings in the notebook,  data cleaning function that are in the `scripts/normalize_data.py`, and tests in `test/test_normalize_data.py`

## Results

By making the raw data, processed data, and data cleaning code available, we hope give ocean core drilling researchers the ability review our data cleaning process and customize the data cleaning steps if so desired. 

## Funding

- Award1 = {"agency": "US National Science Foundation", "award_code": "1928362", "award_URL": "https://www.nsf.gov/awardsearch/showAward?AWD_ID=1928362"}
 
 
## Keywords
Include up to 5 keywords, using the template below.

keywords=["data cleaning", "reproducibility", "version control", "testing"]

## Citation

Wai-Yin Kwan, 2021. Data Cleaning with Pandas and Jupyter Notebooks. Accessed 4/14/2022 at https://github.com/wykhuh/eodp_data_cleaning



## Acknowledgements 

The template is licensed under a <a href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License.</a>

# Setup

## Directory structure

- notebooks: Jupyter notebooks
- processed_data: processed data files
- raw_data: raw, unprocessed data files
- scripts: custom data cleaning scripts 
- tests: unit tests written in pytest

## Library import

In [1]:
# Data manipulation
import pandas as pd

# handle file system
from pathlib import Path 

## Local library import


In [2]:
# Include local library paths
import sys
sys.path.append(str(Path.cwd().parent))

# Import local libraries
from scripts.normalize_data import (
    normalize_columns, 
    remove_bracket_text,
    remove_whitespace,
    normalize_expedition_section_cols,
    print_df
)

## declare variables

variables that are referenced by multiple cells in this notebook

In [3]:
normalized_nontaxa_path = Path('..', 'processed_data', 'normalized_nontaxa_list.csv')
normalized_taxa_path = Path('..', 'processed_data',  'normalized_taxa_list.csv')

taxa_list_path = Path('..', 'processed_data', 'drafts', 'taxa_list.csv')
nontaxa_list_path = Path('..', 'processed_data', 'drafts', 'nontaxa_list.csv')


# Use `Path` and `rglob` to get all the CSVs in `clean_data/taxa` directory.
paths = list(Path('..', 'processed_data', 'clean_data', 'taxa').rglob('*.csv'))
len(paths)

10

# Best practices and processing notes

## Use conda to manage dependencies and virtual environments

The package versions are stored environmental.yml so that when other people run the code, they will install the same packages.

We use virtual environments to avoid dependency conflicts.

## Version Control / Git

We use git to track changes in the code and data files.

We found it helpful to make a commit after each data cleaning step in order to make it easier keep track of the data cleaning steps, and undo the certain steps if needed. To undo a data cleaning step that hasn't been commited, use `git restore`. To undo a data cleaning step that has been commited, use `git revert`, `git reset --hard`, or `get rebase`.

## Unit testing

We wrote unit tests for the data cleaning functions in order to minimize the chances that our data cleaning steps would alter the data files in unexpected ways. The tests are in the `./tests` directory. We used pytest for the unit tests.

Since we use the same data cleaning functions in multiple notebooks, we created a separate tests directory instead of putting the tests inside the notebooks.

## Changes to data files are done in code

In order to make the data cleaning process reproducible, all the changes to the data files should be done in code. We do not manually edit the processed files. 

## read files

We used `pandas.read_csv(path, dtype=str)` to read csv and treat all columns as strings. The reason why we used `dtype=str` is because `pandas.read_csv(path)`  will automatically convert the columns to strings, integers, floats, dates. This automatic conversion can change values in unexpected ways such as converting a column with integers and NaN into floats and NaN. 

In [4]:
path = Path('..', 'processed_data', 'clean_data', 'taxa', 'Micropal_CSV_2', '362_U1480E_planktic_forams.csv')

correct integer values.

In [5]:
df = pd.read_csv(path, nrows=5 , dtype=str)
df['Pulleniatina coiling (dextral)']

0    NaN
1     20
2    NaN
3     23
4     35
Name: Pulleniatina coiling (dextral), dtype: object

incorrect float values. pandas automatically converts the integers to floats because of NaNs.

In [6]:
df = pd.read_csv(path, nrows=5)
df['Pulleniatina coiling (dextral)']

0     NaN
1    20.0
2     NaN
3    23.0
4    35.0
Name: Pulleniatina coiling (dextral), dtype: float64

## viewing the dataframe

One thing that we found helpful when data cleaning is to view the dataframe and the total number of rows and columns to check the changes in the dataframe.

`print_df` is a custom function that calls `pd.DataFrame.shape` and `pd.DataFrame.head()`

In [7]:
path = Path('..', 'processed_data', 'clean_data', 'taxa', 'Micropal_CSV_2', '362_U1480E_planktic_forams.csv')
df = pd.read_csv(path, dtype=str)

print_df(df)

(23, 23)


,Sample,Top [cm],Bottom [cm],Top Depth [m],Bottom Depth [m],Datum name,Zone name (short),Zone name,Preservation,Group abundance,...,Globorotalia flexuosa,Globorotalia hessi,Globorotalia tosaensis,Globorotalia tumida,Neogloboquadrina acostaensis (dextral),Neogloboquadrina humerosa,Pulleniatina coiling (dextral),Pulleniatina coiling (sinistral),Pulleniatina finalis,Sphaeroidinella dahiscens sensu lato
0,362-U1480E-1H-CC-PAL-FORAM_CHEN,0,5,7.71,7.76,NaN,NaN,NaN,P [P43],R [A101],...,NaN,NaN,NaN,R [A102],NaN,R [A102],NaN,NaN,NaN,R [A102]
1,362-U1480E-2H-CC-PAL-FORAM_CHEN,0,5,17.54,17.59,NaN,NaN,NaN,P [P43],A [A101],...,R [A102],NaN,NaN,A [A102],NaN,R [A102],20,0,R [A102],A [A102]
2,362-U1480E-3H-1-W 130/132-FORAM_CHEN,0,2,18.6,18.62,NaN,NaN,NaN,P [P43],R [A101],...,NaN,F [A102],NaN,C [A102],NaN,F [A102],NaN,NaN,NaN,A [A102]
3,362-U1480E-3H-2-W 70/72-FORAM_CHEN,0,2,19.5,19.52,"T Globorotalia tosaensis (Gradstein et al., 2012)",PT1a [AO12],Globigerinoides fistulosus (Anthonissen and Og...,P [P43],A [A101],...,NaN,C [A102],R [A102],A [A102],NaN,NaN,23,0,R [A102],F [A102]
4,362-U1480E-3H-3-W 130/132-FORAM_CHEN,0,2,21.6,21.62,T Neogloboquadrina acostaensis (Gradstein et a...,PT1a [AO12],Globigerinoides fistulosus (Anthonissen and Og...,M [P43],A [A101],...,R [A102],NaN,R [A102],A [A102],F [A102],F [A102],35,12,F [A102],NaN


## Basic cleanup pattern

For each data cleanup step, we loop over all the files, create a dataframe for each file, execute some code to clean the data, and then save the revised file.

In [8]:
for path in paths:
    df = pd.read_csv(path, dtype=str)
    
    # code to change file   
    
    df.to_csv(path, index=False)

## View the changed files

After each cleanup step, we use the desktop application [Github Desktop](https://desktop.github.com/) to spot check the changes to the files. 

If the files look ok, make a commit. If the data cleaning step did not act as expected, undo the changes in the data files using `git restore`, update the data cleaning function and the tests, and rerun the data cleaning step. 

![screenshot of Github Desktop showing the changed data files](images/git_diff.png)


# Data processing and analysis

There are many steps needed to clean up the data files. We created a data cleaning functions and separate sections in the notebooks for each step. 

## Basic file cleanup

pandas has methods that can be used to do some basic file cleanup.

- delete dataframe column if all values are NA 

  dropna(axis='columns', how='all', inplace=True) - [pandas.DataFrame.dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)



- delete dataframe row if all values are NA 

  dropna(axis='index', how='all', inplace=True) - [pandas.DataFrame.dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

- remove duplicate rows in dataframe 

  drop_duplicates(inplace=True) - [pandas.DataFrame.drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html)

before cleanup

In [9]:
path = Path('..', 'processed_data', 'clean_data', 'taxa', 'Micropal_CSV_3', '341_planktic_forams_U1417B.csv')

df = pd.read_csv(path, dtype=str)
print_df(df)

(20, 41)


,Sample,Exp,Site,Hole,Core,Core-Sect,Type,Section,A/W,Top [cm],...,Neogloboquadrina pachyderma (sinistral),Neogloboquadrina pachyderma (dextral),"Neogloboquadrina pachyderma A (dextral, inflated form)","Neogloboquadrina pachyderma B (sinistral, inflated form)","Neogloboquadrina pachyderma C (sinistral, compressed form)",Orbulina universa,Comments,Ship File Links,Shore File Links,File Data
0,341-U1417B-2H-CC-PAL,341,U1417,B,2,2-CC,H,CC,PAL,0,...,D [A84],R [A84],NaN,R [A84],NaN,NaN,NaN,NaN,NaN,NaN
1,341-U1417B-4H-CC-PAL,341,U1417,B,4,4-CC,H,CC,PAL,0,...,D [A84],R [A84],NaN,NaN,NaN,NaN,sandy,NaN,NaN,NaN
2,341-U1417B-6H-CC-PAL,341,U1417,B,6,6-CC,H,CC,PAL,0,...,D [A84],NaN,NaN,NaN,NaN,NaN,diatom ooze. pyritilize foram,NaN,NaN,NaN
3,341-U1417B-9H-CC-PAL,341,U1417,B,9,9-CC,H,CC,PAL,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Rich in sandy terrigenous grains with presence...,NaN,NaN,NaN
4,341-U1417B-10H-CC-PAL,341,U1417,B,10,10-CC,H,CC,PAL,0,...,D [A84],R [A84],NaN,NaN,NaN,NaN,"Rich in planktic foraminifers, pebbles, pyrite",NaN,NaN,NaN


after cleanup. The number of rows changed from 20 to 17, and the number of columns changed from 41 to 26.

In [10]:
df.dropna(axis='columns', how='all', inplace=True)  
df.dropna(axis='index', how='all', inplace=True)
df.drop_duplicates(inplace=True)

print_df(df)

(17, 26)


,Sample,Exp,Site,Hole,Core,Core-Sect,Type,Section,A/W,Top [cm],...,Preservation,Group abundance,Globigerina umbilicata,Globigerinita glutinata,Neogloboquadrina inglei,Neogloboquadrina kagaensis,Neogloboquadrina pachyderma (sinistral),Neogloboquadrina pachyderma (dextral),"Neogloboquadrina pachyderma B (sinistral, inflated form)",Comments
0,341-U1417B-2H-CC-PAL,341,U1417,B,2,2-CC,H,CC,PAL,0,...,G [P15],R [A83],D [A84],R [A84],NaN,NaN,D [A84],R [A84],R [A84],NaN
1,341-U1417B-4H-CC-PAL,341,U1417,B,4,4-CC,H,CC,PAL,0,...,G [P15],C [A61],A [A84],P [A84],NaN,NaN,D [A84],R [A84],NaN,sandy
2,341-U1417B-6H-CC-PAL,341,U1417,B,6,6-CC,H,CC,PAL,0,...,P [P15],P [A83],D [A84],NaN,NaN,NaN,D [A84],NaN,NaN,diatom ooze. pyritilize foram
3,341-U1417B-9H-CC-PAL,341,U1417,B,9,9-CC,H,CC,PAL,0,...,NaN,B [A83],NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rich in sandy terrigenous grains with presence...
4,341-U1417B-10H-CC-PAL,341,U1417,B,10,10-CC,H,CC,PAL,0,...,G [P15],A [A83],D [A84],P [A84],P [A84],NaN,D [A84],R [A84],NaN,"Rich in planktic foraminifers, pebbles, pyrite"


Clean up all files, and save the changes.

In [11]:
for path in paths:
    df = pd.read_csv(path, dtype=str)
    
    df.dropna(axis='columns', how='all', inplace=True)  
    df.dropna(axis='index', how='all', inplace=True)
    df.drop_duplicates(inplace=True)
    
    df.to_csv(path, index=False)

## remove leading and trailing white spaces

We created a custom function `remove_whitespace` to remove all leading and trailing white spaces from both the headers and row values. 

Since we wanted to remove white spaces from both the headers and row values, we used `read_csv(header=None)` and `to_csv(header=False)` so that pandas treat the first row like any other row.

In [12]:
df = pd.read_csv(paths[0], dtype=str, header=None)

remove_whitespace(df)

print_df(df)

(2, 16)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Label ID,Exp,Site,Hole,Core,Core-Sect,Type,Section,A/W,Extra Sample ID Data,Top [cm],Bottom [cm],Top Depth [m],Bottom Depth [m],Globigerina bulloides,Neogloboquadrina cf. pachyderma
1,318-U1355A-1R-1-PAL-FORAM,318,U1355,A,1,1-1,R,1,PAL,FORAM,0,1,0,0.01,R[318_PF],R[318_PF]


remove white space from all files

In [13]:
for path in paths:
    df = pd.read_csv(path, dtype=str, header=None)
    
    remove_whitespace(df)
    
    df.to_csv(path, index=False, header=False)

## Normalizing columns names

For the expedition 312 and later, the researchers for each expedition determined the format of their data files. This resulted in a lot of variability in the file columns. We had to standardized the columns names and update data files. For instance, 'Bottom [cm]' and 'Bottom[cm] [cm]' are 'Bottom [cm]' in the processed files.

Another major challenge with parsing the files is that we had to standardized the taxa names. Issues include mispelling, taxa names change over time, inconsistent ways of treating rank modifiers.

### get all unique column names

In order to normalize the header header names, we needed to get all the headers for all the files. 

Since we only need the header names, use `nrow=0` with `read_csv`. 

In [14]:
pd.read_csv(paths[1], dtype=str, nrows=0)


,Sample,Top [cm],Bottom [cm],Top Depth [m],Bottom Depth [m],Zone name (short),Zone name,Additional zone name (short),Additional zone name,Preservation,...,Globoturborotalita decoraperta _T and B,Globoturborotalita rubescens,Sphaeroidinellopsis seminulina _T_ _PL4,Sphaeroidinellopsis kochi _T,Globigerinoidesella fistulosa _T and B_ _Pt1a,Globigerinoides ruber _pink_ T,Globigerinoides extremus _T and B,Globigerinoides obliquus _T,Globigerinella calida _B,Sample comment


We used `pandas.DataFrame.columns()` and python `set` to get all the unique columns fo all the files.

In [15]:
all_columns = set()
for path in paths:
    df = pd.read_csv(path, dtype=str, nrows=0)
    
    all_columns.update(df.columns)
    
len(all_columns)

158

In [16]:
all_columns

{'A/W',
 'Additional zone name',
 'Additional zone name (short)',
 'Bottom Depth [m]',
 'Bottom Depth[m] [m]',
 'Bottom [cm]',
 'Bottom[cm] [cm]',
 'Candeina nitida',
 'Comment (general)',
 'Comments',
 'Core',
 'Core-Sect',
 'Datum name',
 'Dentoglobigerina altispira _T_ _PL5',
 'Dentoglobigerina altispira _T_ _PL5_',
 'Dextral:Sinistral _P. obliquiloculata_',
 'Dextral:Sinistral _P. praecursor_',
 'Dextral:Sinistral _P. primalis_',
 'Exp',
 'Extra Sample ID Data',
 'Fragmentation',
 'Fragmentation rank [auto-pop]',
 'Genus/species (upper zone)',
 'Genus/species lower zone)',
 'Globigerina bulloides',
 'Globigerina cf. woodi',
 'Globigerina falconensis',
 'Globigerina umbilicata',
 'Globigerinella aequilateralis',
 'Globigerinella calida',
 'Globigerinella calida _B',
 'Globigerinella calida _B_',
 'Globigerinella obesa',
 'Globigerinita glutinata',
 'Globigerinita parkerae',
 'Globigerinita uvula',
 'Globigerinoides bulloideus',
 'Globigerinoides conglobatus',
 'Globigerinoides extre

We then manually separate taxa names from other headers so that we could do some more processing on the taxa.

In [17]:
taxa_columns = {
 'Candeina nitida',
 'Dentoglobigerina altispira _T_ _PL5',
 'Dentoglobigerina altispira _T_ _PL5_',
 'Dextral:Sinistral _P. obliquiloculata_',
 'Dextral:Sinistral _P. praecursor_',
 'Dextral:Sinistral _P. primalis_',
 'Fragmentation rank [auto-pop]',
 'Genus/species (upper zone)',
 'Genus/species lower zone)',
 'Globigerina bulloides',
 'Globigerina cf. woodi',
 'Globigerina falconensis',
 'Globigerina umbilicata',
 'Globigerinella aequilateralis',
 'Globigerinella calida',
 'Globigerinella calida _B',
 'Globigerinella calida _B_',
 'Globigerinella obesa',
 'Globigerinita glutinata',
 'Globigerinita parkerae',
 'Globigerinita uvula',
 'Globigerinoides bulloideus',
 'Globigerinoides conglobatus',
 'Globigerinoides extremus _T and B',
 'Globigerinoides extremus _T and B_',
 'Globigerinoides fistulosus',
 'Globigerinoides obliquus _T',
 'Globigerinoides obliquus _T_',
 'Globigerinoides quadrilobatus',
 'Globigerinoides ruber',
 'Globigerinoides ruber (pink)',
 'Globigerinoides ruber (white)',
 'Globigerinoides ruber _pink_ T',
 'Globigerinoides ruber _pink_ _T_',
 'Globigerinoides sacculifer',
 'Globigerinoides sacculifer (without sack)',
 'Globigerinoides tenellus',
 'Globigerinoides trilobus',
 'Globigerinoidesella fistulosa _T and B_ _Pt1a',
 'Globigerinoidesella fistulosa _T and B_ _Pt1a_',
 'Globoconella miozea',
 'Globorotalia (Globoconella) inflata',
 'Globorotalia (Globorotalia) tumida tumida',
 'Globorotalia (Hirsutella) hirsuta',
 'Globorotalia (Hirsutella) scitula',
 'Globorotalia (Truncorotalia) crossaformis',
 'Globorotalia (Truncorotalia) truncatulinoides',
 'Globorotalia anfracta',
 'Globorotalia crassaformis',
 'Globorotalia crassaformis sensu lato',
 'Globorotalia flexuosa',
 'Globorotalia flexuosa _T and B_',
 'Globorotalia hessi',
 'Globorotalia hessi _B_',
 'Globorotalia hirsuta',
 'Globorotalia inflata',
 'Globorotalia limbata _B',
 'Globorotalia limbata _B_',
 'Globorotalia limbata _T_',
 'Globorotalia margaritae _T and B_ _PL3',
 'Globorotalia margaritae _T and B_ _PL3_',
 'Globorotalia menardii',
 'Globorotalia multicamerata _T',
 'Globorotalia multicamerata _T_',
 'Globorotalia plesiotumida _B_ _M13b_',
 'Globorotalia plesiotumida _T',
 'Globorotalia plesiotumida _T_',
 'Globorotalia pseudomiocenica _T_ _PL6',
 'Globorotalia pseudomiocenica _T_ _PL6_',
 'Globorotalia scitula',
 'Globorotalia tosaensis',
 'Globorotalia tosaensis _T and B_ _Pt1b',
 'Globorotalia tosaensis _T and B_ _Pt1b_',
 'Globorotalia truncatulinoides',
 'Globorotalia truncatulinoides _B',
 'Globorotalia truncatulinoides _B_',
 'Globorotalia tumida',
 'Globorotalia tumida _B_ _PL1a_',
 'Globoturborotalita apertura _T and B',
 'Globoturborotalita apertura _T and B_',
 'Globoturborotalita decoraperta _T and B',
 'Globoturborotalita decoraperta _T and B_',
 'Globoturborotalita rubescens',
 'Neogloboquadrina acostaensis',
 'Neogloboquadrina acostaensis (dextral)',
 'Neogloboquadrina cf. pachyderma',
 'Neogloboquadrina dutertrei',
 'Neogloboquadrina humerosa',
 'Neogloboquadrina incompta (dextral)',
 'Neogloboquadrina inglei',
 'Neogloboquadrina kagaensis',
 'Neogloboquadrina nympha',
 'Neogloboquadrina pachyderma (dextral)',
 'Neogloboquadrina pachyderma (sin)',
 'Neogloboquadrina pachyderma (sinistral)',
 'Neogloboquadrina pachyderma B (sinistral, inflated form)',
 'Neogloboquadrina pachyderma(dex)',
 'Orbulina universa',
 'Pulleniatina coiling (dextral)',
 'Pulleniatina coiling (sinistral)',
 'Pulleniatina finalis',
 'Pulleniatina finalis _B',
 'Pulleniatina finalis _B_',
 'Pulleniatina obliquiloculata',
 'Pulleniatina obliquiloculata (D)',
 'Pulleniatina praecursor',
 'Pulleniatina praespectabilis',
 'Pulleniatina primalis  _Tand B',
 'Pulleniatina primalis  _Tand B_',
 'Sphaeroidinella dahiscens sensu lato',
 'Sphaeroidinella dehiscens',
 'Sphaeroidinella dehiscens s.l.',
 'Sphaeroidinella dehiscens sensu lato _B_',
 'Sphaeroidinellopsis kochi _T',
 'Sphaeroidinellopsis kochi _T_',
 'Sphaeroidinellopsis seminulina _T_ _PL4',
 'Sphaeroidinellopsis seminulina _T_ _PL4_',
}

In [18]:
len(taxa_columns)

117

Since both `all_columns` and `taxa_columns` are sets, we can subtract them to get the nontaxa headers.

In [19]:
nontaxa_columns = all_columns - taxa_columns

nontaxa_columns

{'A/W',
 'Additional zone name',
 'Additional zone name (short)',
 'Bottom Depth [m]',
 'Bottom Depth[m] [m]',
 'Bottom [cm]',
 'Bottom[cm] [cm]',
 'Comment (general)',
 'Comments',
 'Core',
 'Core-Sect',
 'Datum name',
 'Exp',
 'Extra Sample ID Data',
 'Fragmentation',
 'Group Abundance',
 'Group abundance',
 'Group abundance (%)',
 'Hole',
 'Label ID',
 'PF Preservation',
 'Preservation',
 'Sample',
 'Sample comment',
 'Section',
 'Site',
 'Top Depth [m]',
 'Top Depth[m] [m]',
 'Top [cm]',
 'Top[cm] [cm]',
 'Type',
 'Type (lower zone)',
 'Upper boundary age av. [Ma]',
 'Zone author (year)',
 'Zone group',
 'Zone name',
 'Zone name (short)',
 'pc_abundance_name_mode',
 'pc_fossil_group',
 'pc_fossil_name',
 'physical_constituent_name'}

In [20]:
len(nontaxa_columns)

41

### create taxa and non-taxa file

We saved the the taxa and nontaxa headers to csv so that we can access them later.

In [21]:
taxa_df = pd.DataFrame(taxa_columns, columns=['verbatim_name'])
taxa_df.sort_values('verbatim_name', inplace=True)

print_df(taxa_df)

(117, 1)


,verbatim_name
116,Candeina nitida
72,Dentoglobigerina altispira _T_ _PL5
5,Dentoglobigerina altispira _T_ _PL5_
81,Dextral:Sinistral _P. obliquiloculata_
95,Dextral:Sinistral _P. praecursor_


In [22]:
taxa_df.to_csv(taxa_list_path, index=False)

In [23]:
non_taxa_df = pd.DataFrame(nontaxa_columns, columns=['field'])
non_taxa_df.sort_values('field', inplace=True)

print_df(non_taxa_df)

(41, 1)


,field
40,A/W
0,Additional zone name
32,Additional zone name (short)
5,Bottom Depth [m]
36,Bottom Depth[m] [m]


In [24]:
non_taxa_df.to_csv(nontaxa_list_path, index=False)

### normalize headers

After the project PIs manually normalized the columns, we need to update the data files with the noramlized columns.

In [25]:
taxa_df = pd.read_csv(normalized_taxa_path, dtype=str)
print_df(taxa_df)

(117, 3)


,verbatim_name,normalized_name,basic_name
0,Candeina nitida,Candeina nitida,Candeina nitida
1,Dentoglobigerina altispira _T_ _PL5,Dentoglobigerina altispira,Dentoglobigerina altispira
2,Dentoglobigerina altispira _T_ _PL5_,Dentoglobigerina altispira,Dentoglobigerina altispira
3,Dextral:Sinistral _P. obliquiloculata_,Pulleniatina obliquiloculata (dextral),Pulleniatina obliquiloculata
4,Dextral:Sinistral _P. obliquiloculata_,Pulleniatina obliquiloculata (sinistral),Pulleniatina obliquiloculata


In [26]:
nontaxa_df = pd.read_csv(normalized_nontaxa_path, dtype=str)
print_df(nontaxa_df)

(44, 2)


,field,normalized_field
0,A/W,A/W
1,Additional zone name,Additional zone name
2,Additional zone name (short),Additional zone name (short)
3,Bottom Depth [m],Bottom Depth [m]
4,Bottom Depth[m] [m],Bottom Depth [m]


create a dictionary that lists the original field name and normalized field name.

In [27]:
taxa_mapping = taxa_df.set_index('verbatim_name').to_dict()['normalized_name']
taxa_mapping

{'Candeina nitida': 'Candeina nitida',
 'Dentoglobigerina altispira _T_ _PL5': 'Dentoglobigerina altispira',
 'Dentoglobigerina altispira _T_ _PL5_': 'Dentoglobigerina altispira',
 'Dextral:Sinistral _P. obliquiloculata_': 'Pulleniatina obliquiloculata (sinistral)',
 'Dextral:Sinistral _P. praecursor_': 'Pulleniatina praecursor (sinistral)',
 'Dextral:Sinistral _P. primalis_': 'Pulleniatina primalis (sinistral)',
 'Globigerina bulloides': 'Globigerina bulloides',
 'Globigerina cf. woodi': 'Globigerina cf. woodi',
 'Globigerina falconensis': 'Globigerina falconensis',
 'Globigerina umbilicata': 'Globigerina umbilicata',
 'Globigerinella aequilateralis': 'Globigerinella aequilateralis',
 'Globigerinella calida': 'Globigerinella calida',
 'Globigerinella calida _B': 'Globigerinella calida',
 'Globigerinella calida _B_': 'Globigerinella calida',
 'Globigerinella obesa': 'Globigerinella obesa',
 'Globigerinita glutinata': 'Globigerinita glutinata',
 'Globigerinita parkerae': 'Globigerinita 

In [28]:
nontaxa_mapping = nontaxa_df.set_index('field').to_dict()['normalized_field']
nontaxa_mapping

{'A/W': 'A/W',
 'Additional zone name': 'Additional zone name',
 'Additional zone name (short)': 'Additional zone name (short)',
 'Bottom Depth [m]': 'Bottom Depth [m]',
 'Bottom Depth[m] [m]': 'Bottom Depth [m]',
 'Bottom [cm]': 'Bottom [cm]',
 'Bottom[cm] [cm]': 'Bottom [cm]',
 'Comment (general)': 'Comments',
 'Comments': 'Comments',
 'Core': 'Core',
 'Core-Sect': 'Core-Sect',
 'Datum name': 'Datum name',
 'Exp': 'Exp',
 'Extra Sample ID Data': 'Extra Sample ID Data',
 'Fragmentation': 'Fragmentation',
 'Fragmentation rank [auto-pop]': 'Fragmentation rank [auto-pop]',
 'Genus/species (upper zone)': 'Genus/species (upper zone)',
 'Genus/species lower zone)': 'Genus/species (lower zone)',
 'Group Abundance': 'Group Abundance',
 'Group abundance': 'Group Abundance',
 'Group abundance (%)': 'Group Abundance',
 'Hole': 'Hole',
 'Label ID': 'Sample',
 'PF Preservation': 'PF Preservation',
 'Preservation': 'Preservation',
 'Sample': 'Sample',
 'Sample comment': 'Comments',
 'Section': 'Sec

`normalize_columns` replaces the original column names with normalized column names. 

For this file, 'Label ID' is changed to 'Sample', 'Neogloboquadrina pachyderma (sin)' to 'Neogloboquadrina pachyderma (sinistral)',  and 'Neogloboquadrina pachyderma(dex)' to 'Neogloboquadrina pachyderma (dextral)'.

In [29]:
df = pd.read_csv(paths[8], dtype=str)    
df.columns


Index(['Label ID', 'Exp', 'Site', 'Hole', 'Core', 'Core-Sect', 'Type',
       'Section', 'A/W', 'Extra Sample ID Data', 'Top [cm]', 'Bottom [cm]',
       'Top Depth [m]', 'Bottom Depth [m]', 'Comments', 'Preservation',
       'Group Abundance', 'Neogloboquadrina pachyderma (sin)',
       'Neogloboquadrina pachyderma(dex)', 'pc_fossil_group',
       'physical_constituent_name', 'pc_fossil_name',
       'pc_abundance_name_mode'],
      dtype='object')

In [30]:
df = pd.read_csv(paths[8], dtype=str) 
normalize_columns(df, taxa_mapping)
normalize_columns(df, nontaxa_mapping)

df.columns

Index(['Sample', 'Exp', 'Site', 'Hole', 'Core', 'Core-Sect', 'Type', 'Section',
       'A/W', 'Extra Sample ID Data', 'Top [cm]', 'Bottom [cm]',
       'Top Depth [m]', 'Bottom Depth [m]', 'Comments', 'Preservation',
       'Group Abundance', 'Neogloboquadrina pachyderma (sinistral)',
       'Neogloboquadrina pachyderma (dextral)', 'pc_fossil_group',
       'physical_constituent_name', 'pc_fossil_name',
       'pc_abundance_name_mode'],
      dtype='object')

normalize columns for all files

In [31]:
for path in paths:
    df = pd.read_csv(path, dtype=str)    
    
    normalize_columns(df, nontaxa_mapping)
    normalize_columns(df, taxa_mapping)
    
    df.to_csv(path, index=False)


## Turn one column into multiple columns 

For some files, `Sample` or `Label ID` column was given, but `Exp, Site, Hole, Core, Type, Section, A/W` columns where not given. All those columns should be in every file.

`normalize_expedition_section_cols` converts `Sample` or `Label ID` into separate `Exp, Site, Hole, Core, Type, Section, A/W` columns. 

Sample: 363-U1483A-1H-2-W 75/77-FORAM  
Exp: 363, Site: U1483, Hole: A, Core: 1, Type: H, Section: 2, A/W: W

In [32]:
for path in paths:
    df = pd.read_csv(path, dtype=str)   
    
    df = normalize_expedition_section_cols(df)
    
    df.to_csv(path, index=False) 

## Clean up row values

For some of the columns such as taxa, the rows contain abundance data that we want to keep and notes in brackets that we want to remove.  `remove_bracket_text` removes the [note] text.

In [33]:
df = pd.read_csv(paths[0], dtype=str)    
print_df(df)

(1, 16)


,Sample,Exp,Site,Hole,Core,Core-Sect,Type,Section,A/W,Extra Sample ID Data,Top [cm],Bottom [cm],Top Depth [m],Bottom Depth [m],Globigerina bulloides,Neogloboquadrina cf. pachyderma
0,318-U1355A-1R-1-PAL-FORAM,318,U1355,A,1,1-1,R,1,PAL,FORAM,0,1,0,0.01,R[318_PF],R[318_PF]


In [34]:
df = pd.read_csv(paths[0], dtype=str) 
df = remove_bracket_text(df)
print_df(df)

(1, 16)


,Sample,Exp,Site,Hole,Core,Core-Sect,Type,Section,A/W,Extra Sample ID Data,Top [cm],Bottom [cm],Top Depth [m],Bottom Depth [m],Globigerina bulloides,Neogloboquadrina cf. pachyderma
0,318-U1355A-1R-1-PAL-FORAM,318,U1355,A,1,1-1,R,1,PAL,FORAM,0,1,0,0.01,R,R


In [35]:
for path in paths:
    df = pd.read_csv(path, dtype=str)
    
    df = remove_bracket_text(df)
    
    df.to_csv(path, index=False)

## check if mandatory columns exists

After we completed the data cleaning steps, we want to check if certain mandatory columns are present in all the data files.

In [36]:
required_columns = {
 'A/W',
 'Bottom [cm]',
 'Bottom Depth [m]',
 'Core',
 'Exp',
 'Hole',
 'Sample',
 'Section',
 'Site',
 'Top [cm]',
 'Top Depth [m]',
 'Type'
}

If a file if missing some mandatory columns, print the file name and the missing columns.

In [37]:
for path in paths:
    df = pd.read_csv(path, dtype=str)    
    cols = set(df.columns)
    diff = required_columns - cols
    
    if(len(diff) > 0):
        print(path)
        print(required_columns - cols)
    